In [2]:
from ultralytics import YOLO, RTDETR
import supervision as sv

import cv2
import yaml
import torch
import time
import json
import numpy as np
import pandas as pd
from pathlib import Path
import itertools
from collections import defaultdict, deque

from imutils.video import FileVideoStream
from imutils.video import FPS

from tools.print_info import print_video_info, print_progress, step_message
from tools.video_info import from_video_path
from tools.csv_sink import CSVSink
from tools.speed import ViewTransformer

from icecream import ic

In [3]:
# Load input parameters
source: str = 'D:/Data/CCT_Drones/P2_50/P2_50_100_CR.mp4'
output: str = 'D:/Data/CCT_Drones/P2_50'
weights: str = 'D:/Data/models/yolov8/yolov8x_dr_v4'
class_filter: list[int] = []
image_size: int = 640
confidence: float = 0.5
track_length: int = 100
show_image: bool = True
save_csv: bool = True
save_video: bool = True
target_height: int = 100
target_width: int = 24.5

In [4]:
step_count = itertools.count(1)

with open(f"{output}/{Path(source).stem}_output.csv", 'r') as csv_file:
    csv_data = pd.read_csv(csv_file, sep=',', names=['frame', 'id', 'class', 'x', 'y', 'w', 'h', 'score'], header=None, index_col=False)
csv_data



,frame,id,class,x,y,w,h,score
0,0,1,bus,1496,1470,279,138,0.925545
1,0,2,car,602,1470,281,125,0.921028
2,0,3,truck,2963,1431,346,158,0.917146
3,0,4,car,1691,836,229,117,0.906213
4,0,5,truck,0,626,305,134,0.904121
...,...,...,...,...,...,...,...,...
799398,57270,6099,motorcycle,2243,1730,119,58,0.839445
799399,57270,6144,motorcycle,208,1470,118,52,0.828910
799400,57270,6110,motorcycle,1998,178,106,43,0.532192
799401,57270,6139,motorcycle,1705,2013,124,58,0.658281


In [5]:
# Individual object IDs
object_id_list = csv_data['id'].unique().astype(int)
object_id_list

array([   1,    2,    3, ..., 6140, 6142, 6144])

In [6]:
# Processing data
ID_CLASSES = {
    'bicycle': 0,
    'bus': 1,
    'car': 2,
    'motorcycle': 3,
    'person': 4,
    'taxi': 5,
    'truck': 6
}

csv_data['class_id'] = csv_data['class'].replace(ID_CLASSES)
csv_data['x2'] = csv_data['x'] + csv_data['w']
csv_data['y2'] = csv_data['y'] + csv_data['h']
csv_data['time_frame'] = csv_data['frame'] * (1.0 / 59.94)
csv_data['center_x'] = csv_data['x'] + csv_data['w']
csv_data['center_y'] = csv_data['y'] + csv_data['h']
csv_data

,frame,id,class,x,y,w,h,score,class_id,x2,y2,time_frame,center_x,center_y
0,0,1,bus,1496,1470,279,138,0.925545,1,1775,1608,0.000000,1775,1608
1,0,2,car,602,1470,281,125,0.921028,2,883,1595,0.000000,883,1595
2,0,3,truck,2963,1431,346,158,0.917146,6,3309,1589,0.000000,3309,1589
3,0,4,car,1691,836,229,117,0.906213,2,1920,953,0.000000,1920,953
4,0,5,truck,0,626,305,134,0.904121,6,305,760,0.000000,305,760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799398,57270,6099,motorcycle,2243,1730,119,58,0.839445,3,2362,1788,955.455455,2362,1788
799399,57270,6144,motorcycle,208,1470,118,52,0.828910,3,326,1522,955.455455,326,1522
799400,57270,6110,motorcycle,1998,178,106,43,0.532192,3,2104,221,955.455455,2104,221
799401,57270,6139,motorcycle,1705,2013,124,58,0.658281,3,1829,2071,955.455455,1829,2071


In [7]:
csv_data.loc[(csv_data['center_y'] > 415) & (csv_data['center_y'] < 615), 'lane'] = 1
csv_data.loc[(csv_data['center_y'] > 615) & (csv_data['center_y'] < 825), 'lane'] = 2
csv_data.loc[(csv_data['center_y'] > 825) & (csv_data['center_y'] < 1020), 'lane'] = 3
csv_data.loc[(csv_data['center_y'] > 1205) & (csv_data['center_y'] < 1410), 'lane'] = 4
csv_data.loc[(csv_data['center_y'] > 1410) & (csv_data['center_y'] < 1625), 'lane'] = 5
csv_data.loc[(csv_data['center_y'] > 1625) & (csv_data['center_y'] < 1825), 'lane'] = 6
csv_data

,frame,id,class,x,y,w,h,score,class_id,x2,y2,time_frame,center_x,center_y,lane
0,0,1,bus,1496,1470,279,138,0.925545,1,1775,1608,0.000000,1775,1608,5.0
1,0,2,car,602,1470,281,125,0.921028,2,883,1595,0.000000,883,1595,5.0
2,0,3,truck,2963,1431,346,158,0.917146,6,3309,1589,0.000000,3309,1589,5.0
3,0,4,car,1691,836,229,117,0.906213,2,1920,953,0.000000,1920,953,3.0
4,0,5,truck,0,626,305,134,0.904121,6,305,760,0.000000,305,760,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799398,57270,6099,motorcycle,2243,1730,119,58,0.839445,3,2362,1788,955.455455,2362,1788,6.0
799399,57270,6144,motorcycle,208,1470,118,52,0.828910,3,326,1522,955.455455,326,1522,5.0
799400,57270,6110,motorcycle,1998,178,106,43,0.532192,3,2104,221,955.455455,2104,221,NaN
799401,57270,6139,motorcycle,1705,2013,124,58,0.658281,3,1829,2071,955.455455,1829,2071,NaN


In [8]:
object_dict = {
    'id': [],
    'class_id': [],
    'class': [],
    'speed': [],
    'lane': [],
    'trajectory': []
}
for object_number, object in enumerate(object_id_list):
    object_data = csv_data[csv_data['id'] == object].copy()
    
    # Speed Estimation
    average_speed = np.nan
    if len(object_data) > 1:
        previous_time = 0.0
        previous_x = 0
        previous_y = 0
        speed_column = []
        for current_time, current_x, current_y in zip(object_data['time_frame'], object_data['center_x'], object_data['center_y']):
            distance = abs(np.sqrt((current_y-previous_y)**2 + (current_x-previous_x)**2)) * 3.5 / 200
            time_diff = current_time - previous_time
            speed = distance / time_diff * 3.6 if time_diff != 0.0 else 0.0
            speed_column.append(speed)
            previous_time = current_time
            previous_x = current_x
            previous_y = current_y
        object_data['speed'] = speed_column
        average_speed = object_data['speed'][1:].mean()
        
    object_class_id = object_data['class_id'].unique()
    object_class = object_data['class'].unique()
    object_lanes = object_data['lane'].unique().astype(int)
    object_trajectory = object_data[['center_x', 'center_y']].to_numpy()

    if not np.isnan(average_speed) and object_lanes[-1] > 0:
        object_dict['id'].append(object)
        object_dict['class_id'].append(object_class_id[-1])
        object_dict['class'].append(object_class[-1])
        object_dict['speed'].append(average_speed)
        object_dict['lane'].append(object_lanes[-1])
        object_dict['trajectory'].append(object_trajectory)
pd_object_dict = pd.DataFrame(object_dict)
pd_object_dict


,id,class_id,class,speed,lane,trajectory
0,1,1,bus,16.322366,5,"[[1775, 1608], [1782, 1607], [1787, 1608], [17..."
1,2,2,car,15.426642,5,"[[883, 1595], [892, 1595], [898, 1595], [905, ..."
2,3,6,truck,14.504278,5,"[[3309, 1589], [3316, 1589], [3319, 1588], [33..."
3,4,2,car,44.107444,3,"[[1920, 953], [1908, 951], [1898, 951], [1886,..."
4,5,6,truck,40.482308,2,"[[305, 760], [301, 760], [287, 760], [278, 759..."
...,...,...,...,...,...,...
2007,6137,6,truck,25.421031,6,"[[385, 1690], [392, 1692], [403, 1693], [408, ..."
2008,6138,4,person,8.908343,1,"[[1550, 491], [1551, 491], [1547, 491], [1544,..."
2009,6140,4,person,3.376010,1,"[[2255, 540], [2257, 540], [2261, 539], [2262,..."
2010,6142,2,car,19.295624,5,"[[622, 1485], [613, 1483], [614, 1483]]"


In [9]:
# Analysis
count_classes = {
    'person': 0,
    'car': 0,
    'bicycle': 0,
    'motorcycle': 0,
    'bus': 0,
    'truck': 0,
    'taxi': 0
}
speed_list_classes = {
    'person': [],
    'car': [],
    'bicycle': [],
    'motorcycle': [],
    'bus': [],
    'truck': [],
    'taxi': []
}

for index, object in pd_object_dict.iterrows():
    object_id = object['id']
    object_class_id = object['class_id']
    object_class = object['class']
    object_speed = object['speed']
    object_lane = object['lane']

    count_classes[object_class] += 1
    speed_list_classes[object_class].append(object_speed)

count_classes


{'person': 209,
 'car': 429,
 'bicycle': 21,
 'motorcycle': 849,
 'bus': 112,
 'truck': 282,
 'taxi': 110}

In [10]:
speed_list_classes

{'person': [14.424876425978454,
  4.904872308000628,
  6.422568034501946,
  4.81620599613751,
  24.54199656132687,
  8.640918842193445,
  5.340381538504151,
  2.7522825083836464,
  3.7762200000013424,
  17.395709790976685,
  20.940481135795892,
  7.332099356186278,
  6.591399280128874,
  1.781002707870226,
  2.7374991984954153,
  3.1372823013029056,
  1.780127179502142,
  1.8881100000006712,
  3.5274999242465723,
  5.084350555523387,
  2.7724317881549774,
  2.4639715270489857,
  2.724428333700559,
  2.6417414689899097,
  7.0006257255878594,
  2.3103042309036064,
  1.8881099999974547,
  2.517479999999465,
  5.851919462987659,
  7.2127380911163685,
  4.354632615394978,
  9.634333588987689,
  15.230850536386669,
  8.55738192374839,
  3.1651924157499396,
  2.0454524999975106,
  6.0865195055929435,
  6.121736062078818,
  5.186115021275672,
  10.715229491612758,
  3.5621479808874623,
  5.969231448116917,
  17.38245163622729,
  28.274564946853566,
  29.17424376444862,
  6.078743975391168,
  4

In [12]:
with open(f"{output}/{Path(source).stem}.json", 'r') as json_file:
        json_data = json.load(json_file)
        json_network_data = json.dumps(json_data, indent=2).encode('utf-8')

ic(json_network_data)